In [1]:
import os
import json
import pickle
import random
from collections import defaultdict, Counter

from indra.literature.adeft_tools import universal_extract_text
from indra.databases.hgnc_client import get_hgnc_name, get_hgnc_id

from adeft.discover import AdeftMiner
from adeft.gui import ground_with_gui
from adeft.modeling.label import AdeftLabeler
from adeft.modeling.classify import AdeftClassifier
from adeft.disambiguate import AdeftDisambiguator


from adeft_indra.ground.ground import AdeftGrounder
from adeft_indra.model_building.s3 import model_to_s3
from adeft_indra.model_building.escape import escape_filename
from adeft_indra.db.content import get_pmids_for_agent_text, get_pmids_for_entity, \
    get_plaintexts_for_pmids

In [2]:
adeft_grounder = AdeftGrounder()

In [3]:
shortforms = ['GCA']
model_name = ':'.join(sorted(escape_filename(shortform) for shortform in shortforms))
results_path = os.path.abspath(os.path.join('../..', 'results', model_name))

In [4]:
miners = dict()
all_texts = {}
for shortform in shortforms:
    pmids = get_pmids_for_agent_text(shortform)
    if len(pmids) > 10000:
        pmids = random.choices(pmids, k=10000)
    text_dict = get_plaintexts_for_pmids(pmids, contains=shortforms)
    text_dict = {pmid: text for pmid, text in text_dict.items() if len(text) > 5}
    miners[shortform] = AdeftMiner(shortform)
    miners[shortform].process_texts(text_dict.values())
    all_texts.update(text_dict)

longform_dict = {}
for shortform in shortforms:
    longforms = miners[shortform].get_longforms()
    longforms = [(longform, count, score) for longform, count, score in longforms
                 if count*score > 2]
    longform_dict[shortform] = longforms
    
combined_longforms = Counter()
for longform_rows in longform_dict.values():
    combined_longforms.update({longform: count for longform, count, score
                               in longform_rows})
grounding_map = {}
names = {}
for longform in combined_longforms:
    groundings = adeft_grounder.ground(longform)
    if groundings:
        grounding = groundings[0]['grounding']
        grounding_map[longform] = grounding
        names[grounding] = groundings[0]['name']
longforms, counts = zip(*combined_longforms.most_common())
pos_labels = []

In [5]:
list(zip(longforms, counts))

[('giant cell arteritis', 194),
 ('gastric cardia adenocarcinoma', 45),
 ('golgicide a', 31),
 ('glycocholic acid', 25),
 ('general cognitive ability', 24),
 ('general combining ability', 15),
 ('gastric cardiac adenocarcinoma', 13),
 ('guanylyl cyclase a', 5),
 ('glycyrrhizic acid', 5),
 ('germ cell apoptosis', 3),
 ('granulated coal ash', 3),
 ('gastric cancer', 3),
 ('german cockroach', 3),
 ('gastric cardiac carcinoma', 3)]

In [22]:
grounding_map, names, pos_labels = ground_with_gui(longforms, counts, 
                                                   grounding_map=grounding_map,
                                                   names=names, pos_labels=pos_labels, no_browser=True, port=8891)

INFO: [2020-10-26 02:12:48] indra.ontology.bio.ontology - Loading INDRA bio ontology from cache at /home/ubuntu/.indra/bio_ontology/1.4/bio_ontology.pkl


In [23]:
result = [grounding_map, names, pos_labels]

In [24]:
result

[{'gastric cancer': 'NCIT:C5247',
  'gastric cardia adenocarcinoma': 'NCIT:C5247',
  'gastric cardiac adenocarcinoma': 'NCIT:C5247',
  'gastric cardiac carcinoma': 'NCIT:C5247',
  'general cognitive ability': 'ungrounded',
  'general combining ability': 'ungrounded',
  'germ cell apoptosis': 'ungrounded',
  'german cockroach': 'MESH:D020048',
  'giant cell arteritis': 'MESH:D013700',
  'glycocholic acid': 'CHEBI:CHEBI:17687',
  'glycyrrhizic acid': 'MESH:D019695',
  'golgicide a': 'CHEBI:CHEBI:139037',
  'granulated coal ash': 'ungrounded',
  'guanylyl cyclase a': 'HGNC:7946'},
 {'NCIT:C5247': 'Gastric Cardia Adenocarcinoma',
  'MESH:D020048': 'Blattellidae',
  'MESH:D013700': 'Giant Cell Arteritis',
  'CHEBI:CHEBI:17687': 'glycocholic acid',
  'MESH:D019695': 'Glycyrrhizic Acid',
  'CHEBI:CHEBI:139037': 'golgicide A',
  'HGNC:7946': 'NPR1'},
 ['CHEBI:CHEBI:139037',
  'CHEBI:CHEBI:17687',
  'HGNC:7946',
  'MESH:D013700',
  'MESH:D019695',
  'MESH:D020048',
  'NCIT:C5247']]

In [14]:
grounding_map, names, pos_labels = [{'gastric cancer': 'NCIT:C5247',
  'gastric cardia adenocarcinoma': 'NCIT:C5247',
  'gastric cardiac adenocarcinoma': 'NCIT:C5247',
  'gastric cardiac carcinoma': 'NCIT:C5247',
  'general cognitive ability': 'ungrounded',
  'general combining ability': 'ungrounded',
  'germ cell apoptosis': 'ungrounded',
  'german cockroach': 'MESH:D020048',
  'giant cell arteritis': 'MESH:D013700',
  'glycocholic acid': 'CHEBI:CHEBI:17687',
  'glycyrrhizic acid': 'MESH:D019695',
  'golgicide a': 'CHEBI:CHEBI:139037',
  'granulated coal ash': 'ungrounded',
  'guanylyl cyclase a': 'HGNC:7943'},
 {'NCIT:C5247': 'Gastric Cardia Adenocarcinoma',
  'MESH:D020048': 'Blattellidae',
  'MESH:D013700': 'Giant Cell Arteritis',
  'CHEBI:CHEBI:17687': 'glycocholic acid',
  'MESH:D019695': 'Glycyrrhizic Acid',
  'CHEBI:CHEBI:139037': 'golgicide A',
  'HGNC:7943': 'NPR1'},
 ['CHEBI:CHEBI:139037',
  'CHEBI:CHEBI:17687',
  'HGNC:7943',
  'MESH:D013700',
  'MESH:D019695',
  'MESH:D020048',
  'NCIT:C5247']]

In [15]:
excluded_longforms = []

In [16]:
grounding_dict = {shortform: {longform: grounding_map[longform] 
                              for longform, _, _ in longforms if longform in grounding_map
                              and longform not in excluded_longforms}
                  for shortform, longforms in longform_dict.items()}
result = [grounding_dict, names, pos_labels]

if not os.path.exists(results_path):
    os.mkdir(results_path)
with open(os.path.join(results_path, f'{model_name}_preliminary_grounding_info.json'), 'w') as f:
    json.dump(result, f)

In [17]:
additional_entities = {'HGNC:7943': ['NPR1', ['GCA', 'guanylate cyclase']]}

In [18]:
unambiguous_agent_texts = {}

In [19]:
labeler = AdeftLabeler(grounding_dict)
corpus = labeler.build_from_texts((text, pmid) for pmid, text in all_texts.items())
agent_text_pmid_map = defaultdict(list)
for text, label, id_ in corpus:
    agent_text_pmid_map[label].append(id_)

entity_pmid_map = {entity: set(get_pmids_for_entity(*entity.split(':', maxsplit=1),
                                                major_topic=True))for entity in additional_entities}

In [20]:
intersection1 = []
for entity1, pmids1 in entity_pmid_map.items():
    for entity2, pmids2 in entity_pmid_map.items():
        intersection1.append((entity1, entity2, len(pmids1 & pmids2)))

In [21]:
intersection2 = []
for entity1, pmids1 in agent_text_pmid_map.items():
    for entity2, pmids2 in entity_pmid_map.items():
        intersection2.append((entity1, entity2, len(set(pmids1) & pmids2)))

In [22]:
intersection1

[('HGNC:7943', 'HGNC:7943', 91)]

In [23]:
intersection2

[('CHEBI:CHEBI:17687', 'HGNC:7943', 0),
 ('MESH:D013700', 'HGNC:7943', 0),
 ('MESH:D019695', 'HGNC:7943', 0),
 ('HGNC:7943', 'HGNC:7943', 0),
 ('MESH:D020048', 'HGNC:7943', 0),
 ('NCIT:C5247', 'HGNC:7943', 0),
 ('CHEBI:CHEBI:139037', 'HGNC:7943', 0),
 ('ungrounded', 'HGNC:7943', 0)]

In [24]:
all_used_pmids = set()
for entity, agent_texts in unambiguous_agent_texts.items():
    used_pmids = set()
    for agent_text in agent_texts[1]:
        pmids = set(get_pmids_for_agent_text(agent_text))
        new_pmids = list(pmids - all_texts.keys() - used_pmids)
        text_dict = get_plaintexts_for_pmids(new_pmids, contains=agent_texts)
        corpus.extend([(text, entity, pmid) for pmid, text in text_dict.items() if len(text) >= 5])
        used_pmids.update(new_pmids)
    all_used_pmids.update(used_pmids)
        
for entity, pmids in entity_pmid_map.items():
    new_pmids = list(set(pmids) - all_texts.keys() - all_used_pmids)
    if len(new_pmids) > 10000:
        new_pmids = random.choices(new_pmids, k=10000)
    _, contains = additional_entities[entity]
    text_dict = get_plaintexts_for_pmids(new_pmids, contains=contains)
    corpus.extend([(text, entity, pmid) for pmid, text in text_dict.items() if len(text) >= 5])

In [25]:
names.update({key: value[0] for key, value in additional_entities.items()})
names.update({key: value[0] for key, value in unambiguous_agent_texts.items()})
pos_labels = list(set(pos_labels) | additional_entities.keys() |
                  unambiguous_agent_texts.keys())

In [26]:
%%capture

classifier = AdeftClassifier(shortforms, pos_labels=pos_labels, random_state=1729)
param_grid = {'C': [100.0], 'max_features': [10000]}
texts, labels, pmids = zip(*corpus)
classifier.cv(texts, labels, param_grid, cv=5, n_jobs=5)

INFO: [2020-10-29 04:44:29] /adeft/PP/adeft/adeft/modeling/classify.py - Beginning grid search in parameter space:
{'C': [100.0], 'max_features': [10000]}
INFO: [2020-10-29 04:44:34] /adeft/PP/adeft/adeft/modeling/classify.py - Best f1 score of 0.9439207949389019 found for parameter values:
{'logit__C': 100.0, 'tfidf__max_features': 10000}


In [27]:
classifier.stats

{'label_distribution': {'CHEBI:CHEBI:17687': 16,
  'MESH:D013700': 120,
  'MESH:D019695': 5,
  'HGNC:7943': 15,
  'MESH:D020048': 1,
  'NCIT:C5247': 47,
  'CHEBI:CHEBI:139037': 18,
  'ungrounded': 23},
 'f1': {'mean': 0.943921, 'std': 0.05547},
 'precision': {'mean': 0.95176, 'std': 0.046666},
 'recall': {'mean': 0.946364, 'std': 0.053612},
 'NCIT:C5247': {'f1': {'mean': 0.948235, 'std': 0.07754},
  'pr': {'mean': 0.911111, 'std': 0.129577},
  'rc': {'mean': 1.0, 'std': 0.0}},
 'CHEBI:CHEBI:139037': {'f1': {'mean': 0.971429, 'std': 0.057143},
  'pr': {'mean': 0.95, 'std': 0.1},
  'rc': {'mean': 1.0, 'std': 0.0}},
 'MESH:D013700': {'f1': {'mean': 0.96105, 'std': 0.031404},
  'pr': {'mean': 1.0, 'std': 0.0},
  'rc': {'mean': 0.926748, 'std': 0.057152}},
 'ungrounded': {'f1': {'mean': 0.96, 'std': 0.08},
  'pr': {'mean': 1.0, 'std': 0.0},
  'rc': {'mean': 0.933333, 'std': 0.133333}},
 'CHEBI:CHEBI:17687': {'f1': {'mean': 0.931429, 'std': 0.085905},
  'pr': {'mean': 0.883333, 'std': 0.1452

In [28]:
disamb = AdeftDisambiguator(classifier, grounding_dict, names)

In [29]:
disamb.dump(model_name, results_path)

In [30]:
print(disamb.info())

Disambiguation model for GCA

Produces the disambiguations:
	Blattellidae*	MESH:D020048
	Gastric Cardia Adenocarcinoma*	NCIT:C5247
	Giant Cell Arteritis*	MESH:D013700
	Glycyrrhizic Acid*	MESH:D019695
	NPR1*	HGNC:7943
	glycocholic acid*	CHEBI:CHEBI:17687
	golgicide A*	CHEBI:CHEBI:139037

Class level metrics:
--------------------
Grounding                    	Count	F1     
         Giant Cell Arteritis*	120	0.96105
Gastric Cardia Adenocarcinoma*	 47	0.94824
                   Ungrounded	 23	   0.96
                  golgicide A*	 18	0.97143
             glycocholic acid*	 16	0.93143
                         NPR1*	 15	   0.88
            Glycyrrhizic Acid*	  5	    0.8
                 Blattellidae*	  1	    0.0

Weighted Metrics:
-----------------
	F1 score:	0.94392
	Precision:	0.95176
	Recall:		0.94636

* Positive labels
See Docstring for explanation



In [31]:
model_to_s3(disamb)